In [33]:
import numpy as np
import time

**1)**

In [34]:
subsets = [
    {0, 3, 7, 9},
    {1, 4, 8, 11},
    {3, 6, 7, 9},
    {0, 8, 10, 13},
    {0, 10, 12},
    {1, 3, 7, 9, 13},
    {2, 5, 6, 10, 12},
    {1, 3, 7, 9, 13},
    {2, 3, 4, 10, 11}
]
costs = [45, 45, 45, 45, 35, 55, 55, 60, 60]

In [35]:
incidence_mat = np.array([[1 if j in subset else 0 for j in range(14)] for subset in subsets])
print(incidence_mat)
#The incidence matrix is a binary representation used in combinatorial optimization, particularly in crew assignment problems, to depict the association between two sets of entities: tours (or crew assignments) and flights. Each row of the matrix corresponds to a tour, while each column represents a flight.
#In this context:
#If the entry incidence_mat[i][j] is 1, it indicates that flight j is included in tour i.
#If the entry incidence_mat[i][j] is 0, it signifies that flight j is not part of tour i.
#By examining the rows of the incidence matrix, one can ascertain which flights are encompassed by each tour.
#This insight is fundamental for resolving optimization dilemmas where the goal is to allocate crews to flights with the objective of minimizing costs or enhancing efficiency.

[[1 0 0 1 0 0 0 1 0 1 0 0 0 0]
 [0 1 0 0 1 0 0 0 1 0 0 1 0 0]
 [0 0 0 1 0 0 1 1 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 1 0 1 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 1 0 1 0 0 0 1 0 1 0 0 0 1]
 [0 0 1 0 0 1 1 0 0 0 1 0 1 0]
 [0 1 0 1 0 0 0 1 0 1 0 0 0 1]
 [0 0 1 1 1 0 0 0 0 0 1 1 0 0]]


In [36]:
!pip install -q pyomo # Install and import pyomo

In [37]:
from pyomo.environ import*

In [38]:
!apt-get install -y -qq glpk-utils


Let's formulate the Linear Programming (LP) problem based on the given context.

$Objective:$
Minimize the total cost associated with crew assignments.

$Decision Variable:$
Let
$x_i$ be a binary decision variable representing whether tour $i$ is selected $1$ or not $0$, where i ranges from 1 to 8.

LP Formulation:
\begin{align*}
& \text{Minimize} \quad Z = \sum_{i=1}^{8} x_i \cdot \text{costs}[i] \
\\& \text{subject to} \
\\& \sum_{i=1}^{8} \text{incidence_mat}[i][j] \cdot x_i \geq 1 \quad\\ \text
       {for all} \quad j \in {0, 1, \ldots, 13} \
      & x_i \in {0, 1} \\and  \quad \text{for all} \quad i \in {1, 2, \ldots, 8}
\end{align*}

In this formulation, Z represents the total cost of crew assignments to be minimized.

$x_i$ is a binary variable indicating whether tour $i$ is selected $1$ or not $0$.\
$costs$: costs[i] represents the cost associated with tour i.
The first set of constraints ensures that each flight is covered by at least one selected tour.
The second set of constraints defines that $x_i$can only take binary values, denoting whether a tour is selected $1$ or not $0$.
This LP formulation aims to efficiently assign crews to flights while ensuring coverage for all flights and minimizing the total cost of crew assignments.

In [39]:
model = ConcreteModel()
model.x = Var(range(8),domain=Binary)
model.obj = Objective(expr=sum(model.x[i]*costs[i] for i in range(8)), sense=minimize)
model.constraints = ConstraintList()
for i in range(8):
    model.constraints.add(sum(incidence_mat[i][j] * model.x[j] for j in range(8)) >= 1)

In [40]:
solver = SolverFactory('glpk', executable='/usr/bin/glpsol')

In [41]:
start_time = time.time()
result = solver.solve(model)
print(result)
end_time = time.time()
print("Decision Variables:")
for i in range(8):
    print(f"x[{i}] =", value(model.x[i]))


Problem: 
- Name: unknown
  Lower bound: 145.0
  Upper bound: 145.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 8
  Number of nonzeros: 19
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.005255222320556641
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Decision Variables:
x[0] = 1.0
x[1] = 1.0
x[2] = 0.0
x[3] = 0.0
x[4] = 0.0
x[5] = 0.0
x[6] = 1.0
x[7] = 0.0


In [42]:
crew = sum(value(model.x[i]) for i in range(8))
print("Number of crews:", crew)
print("Crews no.:")
for i in range(8):
    if value(model.x[i]) == 1:
        print(f"Crew {i}")

Number of crews: 3.0
Crews no.:
Crew 0
Crew 1
Crew 6


In [43]:
total_cost1 = sum(value(model.x[i]) * costs[i] for i in range(8))
print(total_cost1) #model objective

145.0


**2)**

In [44]:
def greedy_weighted_set_cover(subsets, costs, universal_set):
    I = set()
    subsets_used = []
    for _ in range(len(universal_set)):
        cost_cover_ratio = [(costs[i] / len(subset - I), i) for i, subset in enumerate(subsets) if subset - I]
        if not cost_cover_ratio:
            break
        min_ratio, min_ratio_index = min(cost_cover_ratio)
        I |= subsets[min_ratio_index]
        subsets_used.append(min_ratio_index)
        if I == set(universal_set):
            break
    return subsets_used

start_time1 = time.time()
selected_subsets = greedy_weighted_set_cover(subsets, costs, range(14))
end_time1= time.time()
print("Selected subsets:", selected_subsets)

Selected subsets: [5, 6, 1, 4]


In [45]:
start_time1 = time.time()
selected_subsets = greedy_weighted_set_cover(subsets, costs, range(14))
end_time1 = time.time()
total_cost2 = sum(costs[i] for i in selected_subsets)
print(total_cost2)


190


**3)**

3.1

In [46]:
model1 = ConcreteModel()
model1.x = Var(range(8), domain=Binary)
model1.obj = Objective(expr=sum(model1.x[i] * costs[i] for i in range(8)), sense=minimize)
model1.constraints = ConstraintList()

In [47]:
incidence_mat_nw = [[element for idx, element in enumerate(row) if idx != 5] for row in incidence_mat] # Remove the 6th column (index 5) from incidence_mat
for i in range(8):
    model1.constraints.add(sum(incidence_mat_nw[i][j] * model1.x[j] for j in range(7)) >= 1)
start_time2 = time.time()
result = solver.solve(model1)
end_time2 = time.time()
num_crews_used = sum(value(model1.x[i]) for i in range(8))
print("Number of crews:", num_crews_used)

print("Crews used:")
for i in range(8):
    if value(model1.x[i]) == 1:
        print(f"Crew {i}")

Number of crews: 3.0
Crews used:
Crew 0
Crew 1
Crew 5


In [48]:
total_cost3 = sum(value(model1.x[i]) * costs[i] for i in range(8))
print("Total cost incurred:", total_cost3)

Total cost incurred: 145.0


**3.2**

In [49]:
# new subsets(excluding Chennai to Guwahati)
subsets_without_chennai_to_guwahati = [
    {0, 3, 7, 9},
    {1, 4, 8, 11},
    {3, 6, 7, 9},
    {0, 8, 10, 13},
    {0, 10, 12},
    {1, 3, 7, 9, 13},
    {2, 5, 6, 10, 12},
    {1, 3, 7, 9, 13},
    {2, 3, 4, 10, 11}
]
costs_without_chennai_to_guwahati = [45, 45, 45, 45, 35, 55, 55, 60, 60]

In [50]:
start_time3 = time.time()
subsets_nw = greedy_weighted_set_cover(subsets_without_chennai_to_guwahati, costs_without_chennai_to_guwahati, range(14))
end_time3 = time.time()

total_cost4= sum(costs_without_chennai_to_guwahati[i] for i in subsets_nw)
print(subsets_nw)

[5, 6, 1, 4]


In [51]:
print(total_cost4)

190


**4)**

4.a

In [52]:
ip_runtime = end_time1 - start_time1
greedy_runtime = end_time2 - start_time2
ip_objective = total_cost1
greedy_objective = total_cost4
print("IP Runtime:", ip_runtime, "Greedy Runtime:", greedy_runtime, "IP Objective:", ip_objective, "Greedy Objective:", greedy_objective)


IP Runtime: 0.00019025802612304688 Greedy Runtime: 0.009722471237182617 IP Objective: 145.0 Greedy Objective: 190


4.b

In [53]:
ip_runtime = end_time1 - start_time1
greedy_runtime = end_time2 - start_time2
ip_objective = total_cost3
greedy_objective = total_cost4
print("IP Runtime:", ip_runtime, "Greedy Runtime:", greedy_runtime, "IP Objective:", ip_objective, "Greedy Objective:", greedy_objective)

IP Runtime: 0.00019025802612304688 Greedy Runtime: 0.009722471237182617 IP Objective: 145.0 Greedy Objective: 190
